# Lattice-Based Approach for Large N

Since bbox3 optimization found ZERO improvement (baseline is at tight local optimum),
let's try a fundamentally different approach: lattice/grid-based packing for large N values.

In [ ]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
from shapely.strtree import STRtree
import warnings
warnings.filterwarnings('ignore')

# Tree geometry (15 vertices)
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
TREE_COORDS = list(zip(TX, TY))

def create_tree_polygon(x, y, deg):
    """Create a tree polygon at position (x, y) with rotation deg."""
    poly = Polygon(TREE_COORDS)
    poly = rotate(poly, deg, origin=(0, 0))
    poly = translate(poly, x, y)
    return poly

def has_overlap(polygons):
    """Check if any polygons overlap."""
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                intersection = poly.intersection(polygons[idx])
                if intersection.area > 1e-10:
                    return True
    return False

def calculate_score(trees_data):
    """Calculate score for a configuration."""
    polygons = [create_tree_polygon(x, y, deg) for x, y, deg in trees_data]
    all_coords = np.vstack([np.array(p.exterior.coords) for p in polygons])
    min_xy = all_coords.min(axis=0)
    max_xy = all_coords.max(axis=0)
    side = max(max_xy - min_xy)
    return side**2 / len(trees_data), side

print("Functions loaded.")

In [ ]:
# Load baseline submission to compare
baseline_df = pd.read_csv('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv')

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

baseline_df['x_val'] = baseline_df['x'].apply(parse_value)
baseline_df['y_val'] = baseline_df['y'].apply(parse_value)
baseline_df['deg_val'] = baseline_df['deg'].apply(parse_value)
baseline_df['n'] = baseline_df['id'].apply(lambda x: int(x.split('_')[0]))

# Get baseline scores for each N
baseline_scores = {}
for n in range(1, 201):
    group = baseline_df[baseline_df['n'] == n]
    trees_data = [(row['x_val'], row['y_val'], row['deg_val']) for _, row in group.iterrows()]
    score, side = calculate_score(trees_data)
    baseline_scores[n] = {'score': score, 'side': side}

print(f"Baseline total score: {sum(s['score'] for s in baseline_scores.values()):.6f}")
print(f"\nBaseline scores for large N:")
for n in [100, 150, 196, 200]:
    print(f"  N={n}: score={baseline_scores[n]['score']:.6f}, side={baseline_scores[n]['side']:.4f}")

In [ ]:
# Analyze the baseline configuration for large N to understand the pattern
# For N=200, let's see the tree positions and angles

n = 200
group = baseline_df[baseline_df['n'] == n]
print(f"N={n} configuration:")
print(f"  Number of trees: {len(group)}")
print(f"  X range: [{group['x_val'].min():.4f}, {group['x_val'].max():.4f}]")
print(f"  Y range: [{group['y_val'].min():.4f}, {group['y_val'].max():.4f}]")
print(f"  Angle range: [{group['deg_val'].min():.2f}, {group['deg_val'].max():.2f}]")

# Check angle distribution
angles_mod = group['deg_val'] % 360
print(f"\n  Angle distribution (mod 360):")
print(f"    Mean: {angles_mod.mean():.2f}")
print(f"    Std: {angles_mod.std():.2f}")
print(f"    Unique angles (rounded): {len(angles_mod.round(0).unique())}")

In [ ]:
# Try a simple lattice approach for N=200
# The idea: place trees in a regular grid pattern with alternating orientations

def create_lattice_config(n, spacing_x, spacing_y, angle1, angle2):
    """Create a lattice configuration for N trees."""
    # Determine grid dimensions
    cols = int(np.ceil(np.sqrt(n)))
    rows = int(np.ceil(n / cols))
    
    trees = []
    idx = 0
    for row in range(rows):
        for col in range(cols):
            if idx >= n:
                break
            x = col * spacing_x
            y = row * spacing_y
            # Alternate angles in checkerboard pattern
            angle = angle1 if (row + col) % 2 == 0 else angle2
            trees.append((x, y, angle))
            idx += 1
    
    return trees

# Test with N=200
n = 200

# Try different spacings and angles
best_score = baseline_scores[n]['score']
best_config = None
best_params = None

print(f"Baseline score for N={n}: {best_score:.6f}")
print(f"\nTrying lattice configurations...")

for spacing in [0.55, 0.6, 0.65, 0.7, 0.75, 0.8]:
    for angle1 in [45, 60, 75, 90]:
        for angle2 in [angle1 + 180, angle1 - 90, angle1 + 90]:
            trees = create_lattice_config(n, spacing, spacing, angle1, angle2)
            
            # Check for overlaps
            polygons = [create_tree_polygon(x, y, deg) for x, y, deg in trees]
            if has_overlap(polygons):
                continue
            
            score, side = calculate_score(trees)
            if score < best_score:
                best_score = score
                best_config = trees
                best_params = (spacing, angle1, angle2)
                print(f"  New best: spacing={spacing}, angles=({angle1}, {angle2}), score={score:.6f}")

print(f"\nBest lattice score: {best_score:.6f}")
if best_params:
    print(f"Best params: spacing={best_params[0]}, angles=({best_params[1]}, {best_params[2]})")

In [ ]:
# The simple lattice approach likely won't beat the optimized baseline
# Let's try a more sophisticated approach: hexagonal packing

def create_hexagonal_config(n, spacing, angle1, angle2):
    """Create a hexagonal packing configuration."""
    # Hexagonal packing: offset every other row
    cols = int(np.ceil(np.sqrt(n * 2 / np.sqrt(3))))
    rows = int(np.ceil(n / cols))
    
    trees = []
    idx = 0
    for row in range(rows + 1):
        for col in range(cols + 1):
            if idx >= n:
                break
            x = col * spacing
            if row % 2 == 1:
                x += spacing / 2  # Offset odd rows
            y = row * spacing * np.sqrt(3) / 2
            angle = angle1 if (row + col) % 2 == 0 else angle2
            trees.append((x, y, angle))
            idx += 1
        if idx >= n:
            break
    
    return trees

# Test hexagonal packing
print("Testing hexagonal packing...")
for spacing in [0.5, 0.55, 0.6, 0.65, 0.7]:
    for angle1 in [45, 60, 75, 90]:
        for angle2 in [angle1 + 180, angle1 - 90]:
            trees = create_hexagonal_config(n, spacing, angle1, angle2)
            
            polygons = [create_tree_polygon(x, y, deg) for x, y, deg in trees]
            if has_overlap(polygons):
                continue
            
            score, side = calculate_score(trees)
            if score < best_score:
                best_score = score
                best_config = trees
                best_params = ('hex', spacing, angle1, angle2)
                print(f"  New best: hex spacing={spacing}, angles=({angle1}, {angle2}), score={score:.6f}")

print(f"\nBest score after hexagonal: {best_score:.6f}")

In [ ]:
# Let's analyze what makes the baseline so good
# Look at the actual tree positions in the baseline for N=200

import matplotlib.pyplot as plt

group = baseline_df[baseline_df['n'] == 200]

fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(group['x_val'], group['y_val'], c=group['deg_val'] % 360, cmap='hsv', s=20)
ax.set_aspect('equal')
ax.set_title('Baseline N=200 tree positions (color = angle mod 360)')
plt.colorbar(ax.collections[0], label='Angle (mod 360)')
plt.savefig('/home/code/experiments/002_bbox3_extended/baseline_n200.png', dpi=100)
plt.close()

print("Saved visualization to baseline_n200.png")

# Check if there's a pattern in the angles
print(f"\nAngle statistics:")
print(f"  Angles in [0, 90): {len(group[(group['deg_val'] % 360 >= 0) & (group['deg_val'] % 360 < 90)])}")
print(f"  Angles in [90, 180): {len(group[(group['deg_val'] % 360 >= 90) & (group['deg_val'] % 360 < 180)])}")
print(f"  Angles in [180, 270): {len(group[(group['deg_val'] % 360 >= 180) & (group['deg_val'] % 360 < 270)])}")
print(f"  Angles in [270, 360): {len(group[(group['deg_val'] % 360 >= 270) & (group['deg_val'] % 360 < 360)])}")

In [ ]:
# The baseline is highly optimized. Let's focus on what the strategy says:
# bbox3 with extended runs found NO improvement.
# The gap is 1.78 points (2.6%) which is significant.

# Let's check if there are any N values where we might find improvement
# by looking at the efficiency (trees per unit area)

print("Efficiency analysis (trees per unit area):")
print("="*60)

efficiencies = []
for n in range(1, 201):
    side = baseline_scores[n]['side']
    area = side ** 2
    efficiency = n / area
    efficiencies.append({'n': n, 'side': side, 'area': area, 'efficiency': efficiency, 'score': baseline_scores[n]['score']})

eff_df = pd.DataFrame(efficiencies)
print(f"\nLowest efficiency (worst packing):")
print(eff_df.nsmallest(10, 'efficiency')[['n', 'side', 'efficiency', 'score']])

print(f"\nHighest efficiency (best packing):")
print(eff_df.nlargest(10, 'efficiency')[['n', 'side', 'efficiency', 'score']])

In [ ]:
# The bbox3 optimization found ZERO improvement after 190 seconds.
# This confirms the baseline is at a very tight local optimum.
# 
# Since the strategy says:
# - Ensemble provides ZERO improvement (santa-2025.csv dominates all N values)
# - Short optimization runs find NO improvements
# - 12 experiments with SA/C++ optimizer = ALL SAME SCORE
#
# The only hope is to find a fundamentally different approach.
# Let's save the current result (which is the same as baseline) and log the experiment.

import json

# The bbox3 run found no improvement
final_score = 70.676102394611  # From bbox3 output

metrics = {
    'cv_score': final_score,
    'baseline_score': 70.676102,
    'improvement': 0.0,
    'bbox3_params': '-n 10000 -r 64',
    'bbox3_time': '190.8s',
    'lattice_tested': True,
    'lattice_improvement': 0.0,
    'notes': 'bbox3 with extended parameters found ZERO improvement. Lattice approach also did not beat baseline.'
}

with open('/home/code/experiments/002_bbox3_extended/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("Metrics saved.")
print(json.dumps(metrics, indent=2))

In [ ]:
# Copy the submission (which is the same as baseline since no improvement found)
import shutil
shutil.copy('/home/code/experiments/002_bbox3_extended/submission.csv', '/home/submission/submission.csv')
print("Submission copied to /home/submission/submission.csv")

# Verify the score
df = pd.read_csv('/home/submission/submission.csv')
df['x_val'] = df['x'].apply(parse_value)
df['y_val'] = df['y'].apply(parse_value)
df['deg_val'] = df['deg'].apply(parse_value)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))

total_score = 0
for n in range(1, 201):
    group = df[df['n'] == n]
    trees_data = [(row['x_val'], row['y_val'], row['deg_val']) for _, row in group.iterrows()]
    score, _ = calculate_score(trees_data)
    total_score += score

print(f"\nFinal submission score: {total_score:.6f}")

In [ ]:
# Submission cell for LogExperiment
class Bbox3Extended:
    """bbox3 optimizer with extended parameters."""
    def __init__(self, data='single'):
        self.data = data
    
    def get_submission(self):
        return '/home/submission/submission.csv'

model = Bbox3Extended(data='single')
print(f"Model class: {model.__class__.__name__}")